In [1]:
from typing import List, Tuple
from model import lstm, mlp, bidirectional_lstm, gru
from utils import read_raw_dataset, select_features_name, read_chunk, train_with_data
from fed_algo import fed_avg
from keras.models import Sequential

2023-01-03 22:02:24.065687: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 22:02:24.420726: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-03 22:02:25.471802: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-03 22:02:25.471933: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
X_raw, y = read_raw_dataset("datasets/pokemon.csv",
                            index_col='id',
                            target_name='MOS',
                            drop=['user_id'])
features = select_features_name(X_raw, y)
X_test, y_test = read_chunk('pokemon', '-test', features)
X_init, y_init = read_chunk('pokemon', '-init', features)
result = {
    'num_clients': [25, 50, 75, 100, 125, 150],
    'lstm': [],
    'bidirectional': [],
    'gru': []
}

In [3]:
models = [lstm(), bidirectional_lstm(), gru()]
models_name = ['lstm', 'bidirectional', 'gru']
for model in models:
    train_with_data(model, model.get_weights(), (X_init, y_init))

2023-01-03 22:02:27.282605: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-03 22:02:27.326938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-03 22:02:27.327121: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-03 22:02:27.327864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them i

Epoch 1/20


/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]
2023-01-03 22:02:34.238466: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2023-01-03 22:02:34.458865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-01-03 22:02:34.464030: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1e8a2850 initialized for pla

1/1 [==============================] - 6s 6s/step - loss: 8.6368 - mean_squared_error: 8.6368 - val_loss: 7.2836 - val_mean_squared_error: 7.2836
Epoch 2/20
1/1 [==============================] - 0s 42ms/step - loss: 5.6825 - mean_squared_error: 5.6825 - val_loss: 4.1792 - val_mean_squared_error: 4.1792
Epoch 3/20
1/1 [==============================] - 0s 43ms/step - loss: 3.4258 - mean_squared_error: 3.4258 - val_loss: 1.8735 - val_mean_squared_error: 1.8735
Epoch 4/20
1/1 [==============================] - 0s 41ms/step - loss: 1.7204 - mean_squared_error: 1.7204 - val_loss: 0.6115 - val_mean_squared_error: 0.6115
Epoch 5/20
1/1 [==============================] - 0s 42ms/step - loss: 1.4128 - mean_squared_error: 1.4128 - val_loss: 0.5186 - val_mean_squared_error: 0.5186
Epoch 6/20
1/1 [==============================] - 0s 42ms/step - loss: 2.6340 - mean_squared_error: 2.6340 - val_loss: 0.6568 - val_mean_squared_error: 0.6568
Epoch 7/20
1/1 [==============================] - 0s 45ms/s

/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 3s 3s/step - loss: 9.2512 - mean_squared_error: 9.2512 - val_loss: 4.5082 - val_mean_squared_error: 4.5082
Epoch 2/20
1/1 [==============================] - 0s 50ms/step - loss: 3.7478 - mean_squared_error: 3.7478 - val_loss: 1.0265 - val_mean_squared_error: 1.0265
Epoch 3/20
1/1 [==============================] - 0s 51ms/step - loss: 1.3538 - mean_squared_error: 1.3538 - val_loss: 0.4439 - val_mean_squared_error: 0.4439
Epoch 4/20
1/1 [==============================] - 0s 50ms/step - loss: 2.2325 - mean_squared_error: 2.2325 - val_loss: 0.8401 - val_mean_squared_error: 0.8401
Epoch 5/20
1/1 [==============================] - 0s 50ms/step - loss: 3.0487 - mean_squared_error: 3.0487 - val_loss: 0.6468 - val_mean_squared_error: 0.6468
Epoch 6/20
1/1 [==============================] - 0s 54ms/step - loss: 2.9747 - mean_squared_error: 2.9747 - val_loss: 0.3450 - val_mean_squared_error: 0.3450
Epoch 7/20
1/1 [==============================] - 0s 50ms/s

/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 2s 2s/step - loss: 8.3881 - mean_squared_error: 8.3881 - val_loss: 7.2039 - val_mean_squared_error: 7.2039
Epoch 2/20
1/1 [==============================] - 0s 46ms/step - loss: 5.2305 - mean_squared_error: 5.2305 - val_loss: 4.3767 - val_mean_squared_error: 4.3767
Epoch 3/20
1/1 [==============================] - 0s 44ms/step - loss: 3.1303 - mean_squared_error: 3.1303 - val_loss: 2.1312 - val_mean_squared_error: 2.1312
Epoch 4/20
1/1 [==============================] - 0s 47ms/step - loss: 1.6422 - mean_squared_error: 1.6422 - val_loss: 0.7211 - val_mean_squared_error: 0.7211
Epoch 5/20
1/1 [==============================] - 0s 49ms/step - loss: 1.1015 - mean_squared_error: 1.1015 - val_loss: 0.4220 - val_mean_squared_error: 0.4220
Epoch 6/20
1/1 [==============================] - 0s 45ms/step - loss: 1.7313 - mean_squared_error: 1.7313 - val_loss: 0.6245 - val_mean_squared_error: 0.6245
Epoch 7/20
1/1 [==============================] - 0s 47ms/s

In [4]:
num_clients = [25, 50, 75, 100, 125, 150]

for num_client in num_clients:

    datasets = [
        read_chunk('pokemon',
                   chunk_id + 1 + num_client - 25, features)
        for chunk_id in range(25)
    ]

    for name, model in zip(models_name, models):
        old_weights = model.get_weights()
        local_models = [
            train_with_data(model, old_weights, dataset)
            for dataset in datasets
        ]
        new_weights = fed_avg(local_models)
        model.set_weights(new_weights)
        result[name].append(model.evaluate(X_test, y_test)[0])

Epoch 1/20
1/1 [==============================] - 0s 86ms/step - loss: 1.1171 - mean_squared_error: 1.1171 - val_loss: 0.0245 - val_mean_squared_error: 0.0245
Epoch 2/20
1/1 [==============================] - 0s 46ms/step - loss: 0.5253 - mean_squared_error: 0.5253 - val_loss: 0.1312 - val_mean_squared_error: 0.1312
Epoch 3/20


/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 0s 47ms/step - loss: 2.3537 - mean_squared_error: 2.3537 - val_loss: 0.1784 - val_mean_squared_error: 0.1784
Epoch 4/20
1/1 [==============================] - 0s 40ms/step - loss: 0.9461 - mean_squared_error: 0.9461 - val_loss: 0.2427 - val_mean_squared_error: 0.2427
Epoch 5/20
1/1 [==============================] - 0s 41ms/step - loss: 3.3034 - mean_squared_error: 3.3034 - val_loss: 0.1565 - val_mean_squared_error: 0.1565
Epoch 6/20
1/1 [==============================] - 0s 42ms/step - loss: 2.1528 - mean_squared_error: 2.1528 - val_loss: 0.0352 - val_mean_squared_error: 0.0352
Epoch 7/20
1/1 [==============================] - 0s 41ms/step - loss: 0.8331 - mean_squared_error: 0.8331 - val_loss: 1.8160e-04 - val_mean_squared_error: 1.8160e-04
Epoch 8/20
1/1 [==============================] - 0s 43ms/step - loss: 1.4899 - mean_squared_error: 1.4899 - val_loss: 0.0232 - val_mean_squared_error: 0.0232
Epoch 9/20
1/1 [==============================] -

/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 0s 41ms/step - loss: 1.6501 - mean_squared_error: 1.6501 - val_loss: 2.1065 - val_mean_squared_error: 2.1065
Epoch 4/20
1/1 [==============================] - 0s 41ms/step - loss: 1.5634 - mean_squared_error: 1.5634 - val_loss: 2.2411 - val_mean_squared_error: 2.2411
Epoch 5/20
1/1 [==============================] - 0s 42ms/step - loss: 1.5557 - mean_squared_error: 1.5557 - val_loss: 2.4095 - val_mean_squared_error: 2.4095
Epoch 6/20
1/1 [==============================] - 0s 41ms/step - loss: 1.5979 - mean_squared_error: 1.5979 - val_loss: 2.4552 - val_mean_squared_error: 2.4552
Epoch 7/20
1/1 [==============================] - 0s 38ms/step - loss: 2.0229 - mean_squared_error: 2.0229 - val_loss: 2.6194 - val_mean_squared_error: 2.6194
Epoch 8/20
1/1 [==============================] - 0s 40ms/step - loss: 0.8262 - mean_squared_error: 0.8262 - val_loss: 2.5959 - val_mean_squared_error: 2.5959
Epoch 9/20
1/1 [==============================] - 0s 42ms

...

1/1 [==============================] - 0s 75ms/step - loss: 0.5321 - mean_squared_error: 0.5321 - val_loss: 0.3333 - val_mean_squared_error: 0.3333
Epoch 3/20
1/1 [==============================] - 0s 72ms/step - loss: 2.4160 - mean_squared_error: 2.4160 - val_loss: 0.3862 - val_mean_squared_error: 0.3862
Epoch 4/20
1/1 [==============================] - 0s 43ms/step - loss: 1.0946 - mean_squared_error: 1.0946 - val_loss: 0.3747 - val_mean_squared_error: 0.3747
Epoch 5/20
1/1 [==============================] - 0s 43ms/step - loss: 0.2848 - mean_squared_error: 0.2848 - val_loss: 0.4593 - val_mean_squared_error: 0.4593
Epoch 6/20
1/1 [==============================] - 0s 53ms/step - loss: 0.4519 - mean_squared_error: 0.4519 - val_loss: 0.5422 - val_mean_squared_error: 0.5422
Epoch 7/20
1/1 [==============================] - 0s 39ms/step - loss: 1.1483 - mean_squared_error: 1.1483 - val_loss: 0.9207 - val_mean_squared_error: 0.9207
Epoch 8/20
1/1 [==============================] - 0s 37ms

/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 0s 38ms/step - loss: 0.6634 - mean_squared_error: 0.6634 - val_loss: 0.0137 - val_mean_squared_error: 0.0137
Epoch 4/20
1/1 [==============================] - 0s 39ms/step - loss: 1.1931 - mean_squared_error: 1.1931 - val_loss: 0.0772 - val_mean_squared_error: 0.0772
Epoch 5/20
1/1 [==============================] - 0s 35ms/step - loss: 0.5062 - mean_squared_error: 0.5062 - val_loss: 0.1583 - val_mean_squared_error: 0.1583
Epoch 6/20
1/1 [==============================] - 0s 38ms/step - loss: 0.5107 - mean_squared_error: 0.5107 - val_loss: 0.2815 - val_mean_squared_error: 0.2815
Epoch 7/20
1/1 [==============================] - 0s 38ms/step - loss: 0.4258 - mean_squared_error: 0.4258 - val_loss: 0.4040 - val_mean_squared_error: 0.4040
Epoch 8/20
1/1 [==============================] - 0s 38ms/step - loss: 1.4687 - mean_squared_error: 1.4687 - val_loss: 0.3361 - val_mean_squared_error: 0.3361
Epoch 9/20
1/1 [==============================] - 0s 36ms

/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 0s 50ms/step - loss: 0.3735 - mean_squared_error: 0.3735 - val_loss: 0.4159 - val_mean_squared_error: 0.4159
Epoch 3/20
1/1 [==============================] - 0s 48ms/step - loss: 0.9419 - mean_squared_error: 0.9419 - val_loss: 0.3975 - val_mean_squared_error: 0.3975
Epoch 4/20
1/1 [==============================] - 0s 41ms/step - loss: 1.0813 - mean_squared_error: 1.0813 - val_loss: 0.3584 - val_mean_squared_error: 0.3584
Epoch 5/20
1/1 [==============================] - 0s 43ms/step - loss: 0.6629 - mean_squared_error: 0.6629 - val_loss: 0.2757 - val_mean_squared_error: 0.2757
Epoch 6/20
1/1 [==============================] - 0s 46ms/step - loss: 0.5421 - mean_squared_error: 0.5421 - val_loss: 0.1628 - val_mean_squared_error: 0.1628
Epoch 7/20
1/1 [==============================] - 0s 70ms/step - loss: 0.3012 - mean_squared_error: 0.3012 - val_loss: 0.1036 - val_mean_squared_error: 0.1036
Epoch 8/20
1/1 [==============================] - 0s 43ms

In [5]:
import json

json_result = json.dumps(result, indent=4)
print(json_result)

{
    "num_clients": [
        25,
        50,
        75,
        100,
        125,
        150
    ],
    "lstm": [
        0.3484609127044678,
        0.3239325284957886,
        0.2677907943725586,
        0.2633548974990845,
        0.2516549825668335,
        0.2486572116613388
    ],
    "bidirectional": [
        0.41513490676879883,
        0.4063614010810852,
        0.35422366857528687,
        0.3340492844581604,
        0.30927181243896484,
        0.33003270626068115
    ],
    "gru": [
        0.2999219596385956,
        0.301344633102417,
        0.2826683521270752,
        0.2811218202114105,
        0.29832935333251953,
        0.2670329809188843
    ]
}
